<a href="https://colab.research.google.com/github/Josogrephy/simple_rag/blob/main/Simple_Resume_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install necessary libraries
!pip install sentence-transformers faiss-cpu pypdf2
# Install the Google Generative AI library
!pip install -q -U google-generativeai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from google.colab import files # To upload files in Colab
import PyPDF2 # For reading PDF files
import io # To handle the uploaded file bytes


# I have attached my resume to colab as an example. You can use any files. If you want, here's my resume: https://drive.google.com/file/d/11aI1QFaMoyFOsn62uwol-luvXVxCSJd0/view?usp=sharing
file_name = "/content/jsian_resume.pdf"

# 2. Extract text from the PDF
raw_text_from_pdf = ""
try:
  with open (file_name, "rb") as f:
    pdf_reader = PyPDF2.PdfReader(f)
    num_pages = len(pdf_reader.pages)
    print(f"Number of pages in PDF: {num_pages}")

    for page_num in range(num_pages):
        page_obj = pdf_reader.pages[page_num]
        raw_text_from_pdf += page_obj.extract_text() + "\n" # Add a newline between pages

  # print("\n--- Extracted Text (first 500 chars) ---")
  # print(raw_text_from_pdf[:500])
  # print("...")

except Exception as e:
  print(f"Error reading PDF: {e}")
  raw_text_from_pdf = None # Ensure it's None if extraction fails

if raw_text_from_pdf and raw_text_from_pdf.strip(): # Check if text was successfully extracted
  # 3. Chunk your document (Important for long documents) We will be chunking it by paragraph

  chunks = [para.strip() for para in raw_text_from_pdf.split('\n\n') if para.strip()]
  if not chunks: # If splitting by '\n\n' yields nothing, try by single newline
      chunks = [line.strip() for line in raw_text_from_pdf.split('\n') if line.strip()]

  if not chunks: # If still no chunks, use the whole text as one chunk (not ideal for long docs)
      chunks = [raw_text_from_pdf.strip()]

  print(f"\nNumber of text chunks created: {len(chunks)}")
  # print("\n--- Sample Chunks ---")
  for i, chunk in enumerate(chunks[:3]): # Print first 3 chunks
    print(f"Chunk {i+1}: {chunk[:100]}...") # Print first 100 chars of chunk

  my_documents = chunks # These are the "documents" we will embed

  # 4. Load a pre-trained Sentence Transformer model
  model = SentenceTransformer('all-MiniLM-L6-v2')

  # 5. Generate embeddings for your document chunks
  print("\nGenerating embeddings for document chunks...")
  document_embeddings = model.encode(my_documents)

  # Check the shape of our embeddings
  print("Shape of document embeddings:", document_embeddings.shape)

  # Save these embeddings for later use.
  np.save("my_document_embeddings.npy", document_embeddings)
  # Also save the processed text chunks if you need to map back to them later
  with open("my_document_chunks.txt", "w", encoding="utf-8") as f:
      for chunk in my_documents:
          f.write(chunk + "\n===\n") # Using a separator

  print("Embeddings generated and saved!")
  print("Text chunks also saved to my_document_chunks.txt")

Number of pages in PDF: 3

Number of text chunks created: 1
Chunk 1: Joseph Sian Gou Wei
siangouweijoseph@gmail.com |+65 92265048 |https://www.linkedin.com/in/joseph-sia...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Generating embeddings for document chunks...
Shape of document embeddings: (1, 384)
Embeddings generated and saved!
Text chunks also saved to my_document_chunks.txt


In [6]:
# Ensure you have run the previous code block or have the necessary libraries and data loaded.

# If you haven't run the previous block in the same session, load the embeddings:
# document_embeddings = np.load("my_document_embeddings.npy")
# model = SentenceTransformer('all-MiniLM-L6-v2') # if needed for query embedding later

# 1. Get the dimensionality of our embeddings
d = document_embeddings.shape[1] # Dimension of embeddings

# 2. Create a FAISS index
# IndexFlatL2 is a basic index that performs exact L2 distance search.
# For very large datasets, you might explore more complex FAISS indexes like IndexIVFFlat.
index = faiss.IndexFlatL2(d)

# 3. Add the document embeddings to the index
index.add(document_embeddings)

# Check if the embeddings are added
print("Number of vectors in the FAISS index:", index.ntotal)

# We can save the FAISS index to disk
faiss.write_index(index, "my_faiss_index.index")

print("FAISS index created, populated, and saved!")

Number of vectors in the FAISS index: 1
FAISS index created, populated, and saved!


In [7]:
import google.generativeai as genai

# --- Configuration ---
# Make sure you have your Gemini API key stored as a secret in Colab.
# Name it 'GEMINI_API_KEY'.
try:
    GEMINI_API_KEY = 'Get your own API Key in GOOGLE AI STUDIO' #Get it at https://aistudio.google.com/
    genai.configure(api_key=GEMINI_API_KEY)
except Exception as e:
    print(f"An error occurred during API key configuration: {e}")
    GEMINI_API_KEY = None


# --- Load pre-requisites (if not already in the environment) ---
# Ensure 'model', 'index', and 'my_documents' are loaded from previous steps.

# If you are in a new session, uncomment and run these:
# model = SentenceTransformer('all-MiniLM-L6-v2')
# document_embeddings = np.load("my_document_embeddings.npy")
# index = faiss.read_index("my_faiss_index.index")
# my_documents = [ # Re-define or load your documents
#     "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.",
#     "The Great Wall of China is a series of fortifications made of stone, brick, tamped earth, wood, and other materials.",
#     "The Colosseum is an oval amphitheatre in the centre of the city of Rome, Italy.",
#     "Generative AI refers to artificial intelligence models capable of generating new content, such as text, images, audio, and video.",
#     "RAG combines retrieval mechanisms with generative models to provide more contextually relevant and accurate responses."
# ]


# 1. Define a user query
user_query = "Why you should hire Joseph Sian?"

# 2. Embed the user query
# It's crucial to use the SAME model for embedding the query as you used for the documents.
query_embedding = model.encode([user_query]) # Pass the query as a list

# 3. Search the FAISS index
k = 2 # Number of top relevant documents to retrieve
distances, indices = index.search(query_embedding, k)

# 'indices' will contain the row numbers of the most similar documents in your original 'my_documents' list.
# 'distances' will contain the corresponding similarity scores (e.g., L2 distances).

print(f"Query: {user_query}")
print(f"Retrieved document indices: {indices}")
print(f"Distances: {distances}")

# 4. Retrieve the actual document content
retrieved_docs_content = [my_documents[i] for i in indices[0]]

print("\n--- Retrieved Documents ---")
for i, doc in enumerate(retrieved_docs_content):
    print(f"Doc {i+1}: {doc}")

# 5. Prepare the context and prompt for Gemini
context_for_llm = "\n\n".join(retrieved_docs_content)

prompt_template = f"""Based ONLY on the following context, answer the question.
If the context doesn't contain the answer, say "I don't have enough information from the provided documents."

Context:
{context_for_llm}

Question: {user_query}

Answer:
"""

print("\n--- Prompt for LLM ---")
print(prompt_template)

# 6. Call the Gemini API (if the API key is available)
if GEMINI_API_KEY:
    try:
        llm_model = genai.GenerativeModel('gemini-1.5-flash-latest') # Or your preferred Gemini model
        response = llm_model.generate_content(prompt_template)

        print("\n--- LLM Response ---")
        print(response.text)
    except Exception as e:
        print(f"\nError during Gemini API call: {e}")
        print("Please ensure your API key is correct and you have API access.")
else:
    print("\nSkipping Gemini API call as API key is not configured.")

Query: Why you should hire Joseph Sian?
Retrieved document indices: [[ 0 -1]]
Distances: [[1.3792911e+00 3.4028235e+38]]

--- Retrieved Documents ---
Doc 1: Joseph Sian Gou Wei
siangouweijoseph@gmail.com |+65 92265048 |https://www.linkedin.com/in/joseph-sian-gou-wei/
Experience
Advertising Solutions Architect, Apps & Measurement Jul. 2022 – Present
Google
•Served as the primary point of contact for professional services in the Malaysia market, enhancing client engagement
and improving service delivery.
•Delivered cross-functional support to Commercial, Product, and Operational teams, enhancing collaboration and
streamlining processes to improve overall efficiency.
•Analyzed clients’ challenges and developed tailored solutions, resulting in improved client satisfaction and enhanced
operational efficiency.
•Developed expertise in App and Measurements products, providing guidance and support to both the internal local
team and clients, resulting in improved product understanding and enhan